In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_334707/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/14 15:17:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/14 15:17:25 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for Failed table
failed_data = {
    'fail_date': ['2018-12-28', '2018-12-29', '2019-01-04', '2019-01-05']
}

# Sample data for Succeeded table
succeeded_data = {
    'success_date': ['2018-12-30', '2018-12-31', '2019-01-01', '2019-01-02', '2019-01-03', '2019-01-06']
}

# Convert date strings to datetime objects
failed_df = pd.DataFrame(failed_data)
failed_df['fail_date'] = pd.to_datetime(failed_df['fail_date'])

succeeded_df = pd.DataFrame(succeeded_data)
succeeded_df['success_date'] = pd.to_datetime(succeeded_df['success_date'])


df_person = spark.createDataFrame(failed_df)
df_person.createOrReplaceTempView("Failed")

df_person = spark.createDataFrame(succeeded_df)
df_person.createOrReplaceTempView("Succeeded")



In [4]:
failed_df

,fail_date
0,2018-12-28
1,2018-12-29
2,2019-01-04
3,2019-01-05


In [5]:
succeeded_df

,success_date
0,2018-12-30
1,2018-12-31
2,2019-01-01
3,2019-01-02
4,2019-01-03
5,2019-01-06


In [18]:
query = """
    with all_day as (
        select
            *
        from (
            select fail_date as date_, 'failed' as state from Failed
            union all
            select success_date as date_, 'success'as state from Succeeded 
        )
        where date_ >= "2019-01-01" and date_ <= "2019-12-31"
        order by date_ asc
    )
    
    select
        state as period_state, 
        min(date_) as start_date,
        max(date_) as end_date
    from (
        select
            *,
            sum(
                mark
            ) over (order by date_ asc) as group_id
        from (
            select 
                *,
                case
                    when lag(state, 1, state) over (order by date_ asc) = state then 0
                    else 1
                end as mark
            from all_day a
        )
    ) group by group_id, state

"""

In [19]:
spark.sql(query).show()

24/03/14 15:37:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/14 15:37:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/14 15:37:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/14 15:37:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/14 15:37:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/14 15:37:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/03/14 1

+------------+-------------------+-------------------+
|period_state|         start_date|           end_date|
+------------+-------------------+-------------------+
|     success|2019-01-01 00:00:00|2019-01-03 00:00:00|
|      failed|2019-01-04 00:00:00|2019-01-05 00:00:00|
|     success|2019-01-06 00:00:00|2019-01-06 00:00:00|
+------------+-------------------+-------------------+



In [30]:
query_1 = """
    with all_day as (
        select
            *
        from (
            select fail_date as date_, 'failed' as state from Failed
            union all
            select success_date as date_, 'success'as state from Succeeded 
        )
        where date_ >= "2019-01-01" and date_ <= "2019-12-31"
        order by date_ asc
    )
    


        select 
            *,
            date_sub(
                date_,
                rank() over (
                    partition by state
                    order by date_
                ) 
            ) as pt
        from all_day a order by date_ asc


    
"""

In [31]:
spark.sql(query_1).show()

+-------------------+-------+----------+
|              date_|  state|        pt|
+-------------------+-------+----------+
|2019-01-01 00:00:00|success|2018-12-31|
|2019-01-02 00:00:00|success|2018-12-31|
|2019-01-03 00:00:00|success|2018-12-31|
|2019-01-04 00:00:00| failed|2019-01-03|
|2019-01-05 00:00:00| failed|2019-01-03|
|2019-01-06 00:00:00|success|2019-01-02|
+-------------------+-------+----------+

